# Week 07 — Segmentation & Simpson’s Paradox (세그먼트/믹스)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 전체 KPI가 좋아졌는데, 실제로는 일부 세그먼트만 좋아진 건 아닌가?
- 채널 믹스 변화가 성과 착시를 만들고 있지 않은가?

## 2) Why this matters (Theory)
- Simpson’s paradox: 그룹별 추세와 전체 추세가 반대가 될 수 있음
- 믹스 변화는 '성장'처럼 보이게 할 수 있음
- 세그먼트별 KPI를 기본으로 보고, 전체는 보조로

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Segment KPI table (country × channel × segment)
✅ 세그먼트별로 전환율/매출/마진을 비교합니다.

In [ ]:
t = (df.assign(revenue=df['revenue'].fillna(0), spend=df['marketing_spend'].fillna(0))
       .groupby(['country','channel','segment'])
       .agg(n=('revenue','size'),
            conv_rate=('conversions','mean'),
            rev_mean=('revenue','mean'),
            margin_mean=('gross_margin','mean'),
            spend_mean=('spend','mean'))
       .reset_index())
display(t.sort_values(['conv_rate','rev_mean'], ascending=False).head(25))

In [ ]:
one_sentence_insight("가장 성과가 좋은 세그먼트 1~2개를 골라 '집중 전략'을 한 문장")

### 5.2 Mix check: share of revenue by channel over time (monthly)
✅ 믹스 변화가 전체 성과를 왜곡할 수 있습니다.

In [ ]:
tmp = df.copy()
tmp['month'] = tmp['date'].dt.to_period('M').dt.to_timestamp()
mrev = (tmp.assign(revenue=tmp['revenue'].fillna(0))
           .groupby(['month','channel'])['revenue'].sum()
           .reset_index())
pivot = mrev.pivot(index='month', columns='channel', values='revenue').fillna(0)
share = pivot.div(pivot.sum(axis=1), axis=0)

share.plot(title="Revenue mix share by channel (monthly)")
plt.ylabel("share")
plt.show()

In [ ]:
one_sentence_insight('믹스 변화가 성과에 어떤 해석 리스크를 만드는지 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
